The goal is to build a binomial logistic regression model and evaluate the model's performance. 

Part 1: EDA & Checking Model Assumptions

Part 2: Model Building and Evaluation

Part 3: Interpreting Model Results

IMPORTS AND DATA LOADING

In [8]:
# Packages for numerics + dataframes
import pandas as pd
import numpy as np

# Packages for visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Packages for Logistic Regression & Confusion Matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, \
recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression

ModuleNotFoundError: No module named 'matplotlib'